# AutoHighlight
This notebook generates video highlights from a video file using Azure AI services.

## Pre-requisites
1. Follow [README](../README.md#configure-azure-ai-service-resource) to create essential resource that will be used in this sample.
2. Install required packages.

In [ ]:
%pip install -r ../requirements.txt

## Load Environment Variables

In [ ]:
from dotenv import load_dotenv
import os
import sys
from pathlib import Path

parent_dir = Path(Path.cwd()).parent
# add the parent directory to the path to use shared modules
sys.path.append(
    str(parent_dir)
)

from python.utility import OpenAIAssistant

load_dotenv(dotenv_path=".env", override=True)

AZURE_AI_SERVICE_ENDPOINT = os.getenv("AZURE_AI_SERVICE_ENDPOINT")
AZURE_AI_SERVICE_API_VERSION = os.getenv("AZURE_AI_SERVICE_API_VERSION", "2025-05-01-preview")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Create an OpenAI Assistant to interact with Azure OpenAI
openai_assistant = OpenAIAssistant(
    aoai_end_point=AZURE_OPENAI_ENDPOINT,
    aoai_api_version=AZURE_OPENAI_API_VERSION,
    deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
    aoai_api_key=None,
)

If you haven't done so, please authenticate by running **'az login'** through the terminal. This credentials are used to validate that you have access to the resources you defined above.

Make sure you have Azure CLI installed on your system. To install:
```bash
curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
```

In [ ]:
## Authehticate if you are running this notebook for the first time.
import subprocess
subprocess.run("az login", shell=True)

## VIDEO CONFIGURATION

Use the following variables to configure the auto-highlight generation tailored to your video content and preferences.

Key settings you control:

- **🎥 Video source**: Path to your video file
- **🎬 Content type**: What kind of video you're analyzing (sports, keynote, etc.)
- **⏱️ Target length**: How long you want your final highlight video
- **🎯 Style preferences**: Clip density, transitions, effects
- **🚀 Personalization**: Anything specific you expect your highlights to pertain to

> Important: You must specify the `SOURCE_VIDEO_PATH` and choose the right `VIDEO_TYPE` before running the notebook. The `VIDEO_TYPE` determines what events the AI looks for (e.g., "soccer" finds goals and exciting plays, "keynote" finds key quotes and audience reactions).

In [ ]:
# 🎥 VIDEO CONFIGURATION

import time
import uuid
# Replace with your actual video file path
SOURCE_VIDEO_PATH = "/workspaces/azure-ai-content-understanding-with-azure-openai-python/data/FlightSimulator.mp4"  # Update this path to your video file

# 📁 OUTPUT DIRECTORY (will create output folder as default)
OUTPUT_DIR = parent_dir / "output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 🎬 SCHEMA CONFIGURATION (schema types available in ./schemas/ folder)
VIDEO_TYPE = "demo"  # Options: "soccer", "basketball", "football", etc.

# Path to the schema file (this will be auto-configured based on VIDEO_TYPE)
ANALYZER_DIR = "/workspaces/azure-ai-content-understanding-with-azure-openai-python/analyzer_templates/auto_highlight_analyzers"

# 🎯 HIGHLIGHT GENERATION PARAMETERS
TARGET_DURATION_S = 20           # Target duration for the final highlight video in seconds
CLIP_DENSITY = 1.0               # Density of clips to include (1.0 = normal, >1.0 = more clips, <1.0 = fewer clips)
PERSONALIZATION = "none"         # Any specific expectations from the highlight
TRANSITION_TYPE = "cut"          # Options: "cut", "fade"
SPEED_RAMP = False               # Whether to apply speed ramping
ADD_CAPTIONS = False             # Whether to add captions (requires additional setup, keep FALSE)
RESOLUTION = 720                 # Output resolution height (720p, 1080p)
HUMAN_IN_THE_LOOP_REVIEW = True # Whether to pause for human review of analyzer generation

# Analyzer ID configuration
ANALYZER_ID = f"highlight-analyzer-{str(uuid.uuid4())[:8]}-{int(time.time())}"  # Unique analyzer ID with UUID


if not os.path.exists(SOURCE_VIDEO_PATH):
    raise ValueError(f"❌ Video file not found: {SOURCE_VIDEO_PATH}")

print("✅ Configuration loaded successfully!")
print("🔧 Configuration Details:")
print(f"Source Video: {SOURCE_VIDEO_PATH}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Analyzer Directory: {ANALYZER_DIR}")
print(f"Video Type: {VIDEO_TYPE}")
print(f"Target Duration: {TARGET_DURATION_S}s")
print(f"Analyzer ID: {ANALYZER_ID}")

## 1. Generate and Activate Schema
*What this does:* Creates a custom Azure AI analyzer that uses OpenAI reasoning to understand your specific video content and identify highlight-worthy moments based on your preferences.

The schema generation process takes your VIDEO_TYPE, CLIP_DENSITY, TARGET_DURATION_S, and PERSONALIZATION settings and builds an intelligent analyzer that knows exactly what to look for in your video (e.g., goals in soccer, key quotes in presentations, exciting moments in gameplay).

In [ ]:
from python.auto_highlight_utility import activate_schema

schema_path = activate_schema(
    video_type=VIDEO_TYPE,
    analyzer_dir=ANALYZER_DIR,
    openai_assistant=openai_assistant,
    output_dir=OUTPUT_DIR,
    clip_density=CLIP_DENSITY,
    target_duration_s=TARGET_DURATION_S, 
    personalization=PERSONALIZATION,
    human_in_the_loop_review=HUMAN_IN_THE_LOOP_REVIEW)

print(f"Schema activated: {schema_path}")

# Display schema content
with open(schema_path, 'r') as f:
    import json
    print(json.dumps(json.load(f), indent=2))

## 2. Analyze Video
Now, we submit the video to Azure Content Understanding (CU) for analysis using the custom schema we just generated. This step can take a long time depending on the length of the video.

### Create a Custom Analyzer

In [ ]:
import json

from python.content_understanding_client import AzureContentUnderstandingClient

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

# Create the Content Understanding (CU) client
cu_client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_SERVICE_ENDPOINT,
    api_version=AZURE_AI_SERVICE_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/video_chapters_dynamic", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

# Use the client to create an analyzer
response = cu_client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=schema_path)
result = cu_client.poll_result(response)

print(json.dumps(result, indent=2))

### Use the created analyzer to extract video content

In [ ]:
# Submit the video for content analysis
response = cu_client.begin_analyze(ANALYZER_ID, file_location=SOURCE_VIDEO_PATH)

# Wait for the analysis to complete and get the content analysis result
video_cu_result = cu_client.poll_result(
    response, timeout_seconds=3600)  # 1 hour timeout for long videos

# Print the content analysis result
print(f"Video Content Understanding result: ", video_cu_result)

cu_result_filename = f"{os.path.basename(SOURCE_VIDEO_PATH)}.json"
cu_result_path = os.path.join(OUTPUT_DIR, cu_result_filename)

# Save results
with open(cu_result_path, 'w', encoding='utf-8') as f:
    json.dump(video_cu_result, f, indent=2)

print(f"Video analysis complete. Results saved to: {cu_result_path}")


In [ ]:
# Skip analysis TODO: remove
cu_result_filename = f"{os.path.basename(SOURCE_VIDEO_PATH)}.json"
cu_result_path = os.path.join(OUTPUT_DIR, cu_result_filename)
cu_client.delete_analyzer(ANALYZER_ID)

### Optional - Delete the analyzer if it is no longer needed

In [ ]:
cu_client.delete_analyzer(ANALYZER_ID)

## 3. Parse and Pre-filter Segments

Now we receive the output from Azure CU, which describes all the video frames and their content. We apply rule-based filtering to remove frames that are not worthy enough to be part of the highlights, based on the schema's scoring criteria. This creates a simplified list of potential highlight clips.

In [ ]:
from python.auto_highlight_utility import get_filtered_segments

prefiltered_segments_path = get_filtered_segments(input_path=cu_result_path)
print(f"Segments parsed and pre-filtered. Results saved to: {prefiltered_segments_path}")

# Display the pre-filtered segments
with open(prefiltered_segments_path, 'r') as f:
    segments_data = json.load(f)
    print(json.dumps(segments_data, indent=2))

## 4. Highlight Edits Filtering
Now we send the parsed output to OpenAI o1 for advanced reasoning and intelligent timestamp selection. The AI analyzes all the potential highlight clips and selects the specific timestamps that will create the most compelling custom highlights catered to your preferences, meeting the `TARGET_DURATION_S` and arranging them in an optimal narrative order.

In [ ]:
from python.utility import get_highlight_plan

result = get_highlight_plan(
    openai_assistant=openai_assistant,
    segments=segments_data,
    video_type=VIDEO_TYPE,
    target_duration_s=TARGET_DURATION_S,
    clip_density=CLIP_DENSITY,
    personalization=PERSONALIZATION,
)

In [ ]:
result_dict = result.model_dump()

# Save to JSON file
highlight_plan_file = "highlight_plan.json"
highlight_plan_path = os.path.join(OUTPUT_DIR, highlight_plan_file)
with open(highlight_plan_path, "w", encoding="utf-8") as f:
    json.dump(result_dict, f, indent=2, ensure_ascii=False)

print(f"Saved JSON to {highlight_plan_path}")

## 5. Stitch Video Clips
From the feedback we received from OpenAI's reasoning, we now work on stitching together the custom user highlights. This final step takes the intelligently selected timestamps and creates a seamless highlight video tailored to your preferences using FFmpeg for video editing.

In [ ]:
# Video Stitching using FFmpeg
from python.video_modification_utility import stitch_video

# Define output path for the highlight video
OUTPUT_HIGHLIGHT_PATH = os.path.join(OUTPUT_DIR, "highlight.mp4")

print("🎬 Running video stitching with FFmpeg...")
print(f"📄 Input video: {SOURCE_VIDEO_PATH}")
print(f"📄 Highlight plan: {highlight_plan_path}")
print(f"📄 Output video: {OUTPUT_HIGHLIGHT_PATH}")

try:
    result = stitch_video(
        video_path=SOURCE_VIDEO_PATH,
        plan_path=highlight_plan_path,
        output_path=OUTPUT_HIGHLIGHT_PATH,
        transition=TRANSITION_TYPE,
        speed_ramp=SPEED_RAMP,
        resolution=RESOLUTION
    )
    
    if result:
        print(f"✅ SUCCESS: Highlight video created at {result}")
        
        # Check file info
        if os.path.exists(result):
            size_mb = os.path.getsize(result) / (1024*1024)
            print(f"   📁 File size: {size_mb:.2f} MB")
            print(f"   📁 Full path: {result}")
        print(f"\n🎉 Highlight generation pipeline completed successfully!")
    else:
        print("❌ Video stitching failed")
        
except Exception as e:
    print(f"❌ ERROR during video stitching: {e}")
    import traceback
    traceback.print_exc()